In [1]:
## 분석에 필요한 모듈 불러오기
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
import os
%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [2]:
# 렌딩클럽 07년-11년 데이터 불러오기
lc_0711 = pd.read_csv('LoanStats3a.csv', engine="python")
# 렌딩클럽 12년-13년 데이터 불러오기
lc_1213 = pd.read_csv("LoanStats3b.csv", engine="python")

In [3]:
lc_0713 = lc_0711.append(lc_1213)

In [6]:
lc_0713.loan_status.unique()

array(['Fully Paid', 'Charged Off', nan,
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Current',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Default'], dtype=object)

In [11]:
lc_0713['new_loan_status'] = lc_0713['loan_status'].replace({
    'Does not meet the credit policy. Status:Fully Paid': 'Fully Paid',
    'Does not meet the credit policy. Status:Charged Off': 'Charged Off',
    'Late (31-120 days)': 'Late',
    'Late (16-30 days)': 'Late',
    'In Grace Period': 'Late',
    'Default': 'Late'
})

In [12]:
lc_0713.new_loan_status.value_counts()

Fully Paid     186685
Charged Off     35487
Current          7955
Late              589
Name: new_loan_status, dtype: int64

In [17]:
lc_0713['new_int_rate'] = lc_0713['int_rate'].str[:-1].astype(np.float)

In [23]:
lc_0713['new_grade'] = lc_0713['grade'].replace({'F':'FG','G':'FG'})

In [78]:
ct = pd.crosstab(lc_0713.new_grade,lc_0713.new_loan_status)
ct

new_loan_status,Charged Off,Current,Fully Paid,Late
new_grade,,,,
A,2227,179,36351,6
B,8363,1072,65545,32
C,10129,3429,44992,199
D,7523,1368,24900,117
E,4323,1148,10026,141
FG,2922,759,4871,94


In [79]:
ct = ct.reindex(columns=['Fully Paid', 'Current', 'Late', 'Charged Off'])
ct

new_loan_status,Fully Paid,Current,Late,Charged Off
new_grade,,,,
A,36351,179,6,2227
B,65545,1072,32,8363
C,44992,3429,199,10129
D,24900,1368,117,7523
E,10026,1148,141,4323
FG,4871,759,94,2922


In [80]:
ct.columns=['FULLY PAID', 'CURRENT', 'LATE', 'CHARGED OFF']
ct

,FULLY PAID,CURRENT,LATE,CHARGED OFF
new_grade,,,,
A,36351,179,6,2227
B,65545,1072,32,8363
C,44992,3429,199,10129
D,24900,1368,117,7523
E,10026,1148,141,4323
FG,4871,759,94,2922


In [72]:
total = lc_0713.groupby('new_grade').count()['loan_amnt']
total = total.reset_index()
total

,new_grade,loan_amnt
0,A,38763
1,B,75012
2,C,58749
3,D,33908
4,E,15638
5,FG,8646


In [99]:
interest = lc_0713.groupby('new_grade')['new_int_rate'].mean()
interest = interest.reset_index()
interest['new_int_rate'] = round(interest['new_int_rate'],2)
interest

,new_grade,new_int_rate
0,A,7.60
1,B,11.76
2,C,15.18
3,D,18.08
4,E,20.65
5,FG,23.09


In [158]:
# Merge 말고 Join 쓰는게 더 깔끔함
result = pd.merge(total, ct, left_on='new_grade',right_index=True)
result = pd.merge(result, interest, on='new_grade')
result

,new_grade,loan_amnt,FULLY PAID,CURRENT,LATE,CHARGED OFF,new_int_rate
0,A,38763,36351,179,6,2227,7.60
1,B,75012,65545,1072,32,8363,11.76
2,C,58749,44992,3429,199,10129,15.18
3,D,33908,24900,1368,117,7523,18.08
4,E,15638,10026,1148,141,4323,20.65
5,FG,8646,4871,759,94,2922,23.09


In [159]:
result = result.set_index('new_grade')

In [160]:
result

,loan_amnt,FULLY PAID,CURRENT,LATE,CHARGED OFF,new_int_rate
new_grade,,,,,,
A,38763,36351,179,6,2227,7.60
B,75012,65545,1072,32,8363,11.76
C,58749,44992,3429,199,10129,15.18
D,33908,24900,1368,117,7523,18.08
E,15638,10026,1148,141,4323,20.65
FG,8646,4871,759,94,2922,23.09


In [161]:
result.columns=["TOTAL ISSUED", "FULLY PAID", "CURRENT", "LATE",
               "CHARGED OFF", "AVG.INTEREST RATE"]
del result.index.name
result

,TOTAL ISSUED,FULLY PAID,CURRENT,LATE,CHARGED OFF,AVG.INTEREST RATE
A,38763,36351,179,6,2227,7.60
B,75012,65545,1072,32,8363,11.76
C,58749,44992,3429,199,10129,15.18
D,33908,24900,1368,117,7523,18.08
E,15638,10026,1148,141,4323,20.65
FG,8646,4871,759,94,2922,23.09


In [162]:
result.loc['All'] = All

In [164]:
result

,TOTAL ISSUED,FULLY PAID,CURRENT,LATE,CHARGED OFF,AVG.INTEREST RATE
A,38763.0,36351.0,179.0,6.0,2227.0,7.60
B,75012.0,65545.0,1072.0,32.0,8363.0,11.76
C,58749.0,44992.0,3429.0,199.0,10129.0,15.18
D,33908.0,24900.0,1368.0,117.0,7523.0,18.08
E,15638.0,10026.0,1148.0,141.0,4323.0,20.65
FG,8646.0,4871.0,759.0,94.0,2922.0,23.09
All,38452.0,31114.0,1325.0,98.0,5914.0,16.06


In [181]:
result.loc[:,'TOTAL ISSUED':'CHARGED OFF'] = result.loc[:,'TOTAL ISSUED':'CHARGED OFF'].apply(lambda g: g.astype(int), axis=1)

In [182]:
result

,TOTAL ISSUED,FULLY PAID,CURRENT,LATE,CHARGED OFF,AVG.INTEREST RATE
A,38763.0,36351.0,179.0,6.0,2227.0,7.60
B,75012.0,65545.0,1072.0,32.0,8363.0,11.76
C,58749.0,44992.0,3429.0,199.0,10129.0,15.18
D,33908.0,24900.0,1368.0,117.0,7523.0,18.08
E,15638.0,10026.0,1148.0,141.0,4323.0,20.65
FG,8646.0,4871.0,759.0,94.0,2922.0,23.09
All,38452.0,31114.0,1325.0,98.0,5914.0,16.06
